<a href="https://colab.research.google.com/github/nanafish/ORS/blob/main/%E6%9D%A5%E5%A0%B4%E8%80%85n%E5%B9%B4%E5%88%86%E3%82%B3%E3%82%A2%E5%95%86%E5%9C%8F%E3%83%9E%E3%83%83%E3%83%94%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# ===============================================
# 郵便番号マッピング → n回分の統合コア商圏（○○位置.csv 版）
# - 会場座標：緯度経度を直指定
# - 2σカット（平均+2σ以内）
# - 出力：点KML / 面KML
# - ★面が変なとこに出る例外に対応：MultiPolygon は複数面として出力（上位N面）
# ===============================================

!pip -q install simplekml shapely alphashape

# ===== 設定（ここだけ書き換え） =====
VISITOR_CSV_LIST = [
    "AJ-41-10-1w-周南地域地場産業振興センター-新規対象来場165／441.csv",
    "AJ-41-5-4w-周南地域地場産業振興センター-ももこ展-eternity-新規対象来場265／566.csv",
    "AJ-42-4 3w-周南地域地場産業振興センター-ももこ版画展-bijou-新規対象来場89／286.csv"
]
VENUE_LAT  = 34.020605847089264
VENUE_LON  = 131.82085530749418
VENUE_NAME = "周南地場産センター"

OUTPUT_KML_POLY   = "周南地場産センター.kml"
OUTPUT_KML_POINTS = "周南地場産センター点.kml"

# ===== ここから下は触らなくてOK =====
import pandas as pd
import math, re, glob, os, zipfile
import numpy as np
import simplekml
import alphashape
from shapely.geometry import Point, MultiPoint, Polygon, MultiPolygon
from shapely.ops import unary_union

# ---------- 共通関数 ----------
def read_csv_auto(path, header_opt):
    encodings = ["cp932", "utf-8-sig", "utf-8"]
    last_err = None
    for enc in encodings:
        try:
            df = pd.read_csv(path, encoding=enc, header=header_opt)
            print(f"{path} を encoding='{enc}' で読み込み成功")
            return df
        except UnicodeDecodeError as e:
            last_err = e
            continue
        except FileNotFoundError as e:
            raise e
    raise UnicodeDecodeError(f"{path}", b"", 0, 1, f"試したエンコーディング {encodings} すべてで失敗: {last_err}")

def make_town_base(t):
    t = str(t)
    t = re.sub(r"（.*?）", "", t)
    t = re.sub(r"([一二三四五六七八九十〇零0-9]+)丁目$", "", t)
    return t.strip()

def haversine_m(lon1, lat1, lon2, lat2):
    R = 6371000.0
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlmb = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlmb/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c

def sigma_cut(df, dist_col="dist_m", k=2.0):
    if df.empty:
        return df, None, None, None
    mu = float(df[dist_col].mean())
    sd = float(df[dist_col].std(ddof=0))
    thr = mu + k * sd
    out = df[df[dist_col] <= thr].copy()
    return out, mu, sd, thr

def build_alpha_shape(coords, target_ratio=0.1):
    if len(coords) < 3:
        return None

    mp_all = MultiPoint(coords)
    convex = mp_all.convex_hull
    convex_area = convex.area

    if convex_area == 0:
        poly = convex
    else:
        alpha_candidates = np.logspace(-2.3, 1, 24)  # ≒ 0.005〜10
        best_alpha = None
        best_shape = None
        best_diff = float("inf")

        for a in alpha_candidates:
            try:
                shp = alphashape.alphashape(coords, a)
            except Exception:
                continue
            if shp.is_empty or getattr(shp, "area", 0) == 0:
                continue

            ratio = shp.area / convex_area
            diff = abs(ratio - target_ratio)
            if diff < best_diff:
                best_diff = diff
                best_alpha = a
                best_shape = shp

        print("  chosen alpha:", best_alpha, " diff_from_target=", best_diff)
        poly = best_shape if best_shape is not None else convex

    # 面になってない場合は薄く面化
    if poly.geom_type in ("LineString", "MultiLineString", "Point", "MultiPoint"):
        poly = poly.buffer(1e-4)

    return poly

# ★MultiPolygon対応：複数面として返す
def hull_to_polygons(hull, max_parts=10, min_area_ratio=0.02):
    """
    hull(Polygon/MultiPolygon/その他) -> [Polygon, ...]
    - MultiPolygon は面積の大きい順に最大max_parts
    - 最大面積に対して min_area_ratio 未満の破片は除外
    - self-intersection等の軽微な不正は buffer(0) で補正
    """
    if hull is None or getattr(hull, "is_empty", True):
        return []

    g = hull

    # 面じゃない場合は面化
    if g.geom_type not in ("Polygon", "MultiPolygon"):
        g = g.buffer(1e-4)

    # 補正（不正ジオメトリ対策）
    try:
        g = g.buffer(0)
    except Exception:
        pass

    polys = []
    if g.geom_type == "Polygon":
        polys = [g]
    elif g.geom_type == "MultiPolygon":
        polys = list(g.geoms)
    else:
        return []

    # 面積順
    polys = [p for p in polys if (p is not None and (not p.is_empty) and p.area > 0)]
    if not polys:
        return []

    polys.sort(key=lambda x: x.area, reverse=True)
    max_area = polys[0].area

    # 小破片除外（最大面積比）
    if max_area > 0:
        polys = [p for p in polys if (p.area / max_area) >= float(min_area_ratio)]

    # 上限
    polys = polys[:int(max_parts)]

    # 念のためPolygon化（バッファで崩れた場合対策）
    out = []
    for p in polys:
        try:
            p2 = p.buffer(0)
        except Exception:
            p2 = p
        if p2.geom_type == "Polygon" and (not p2.is_empty) and p2.area > 0:
            out.append(p2)
        elif p2.geom_type == "MultiPolygon":
            # 補正でMultiになったら最大を採用して崩壊回避
            gg = max(list(p2.geoms), key=lambda x: x.area)
            if gg.geom_type == "Polygon" and gg.area > 0:
                out.append(gg)

    return out

# ★穴（interiors）があれば innerboundaryis も付ける（Noneは渡さない）
def add_polygon(kml_obj, poly: Polygon, name: str, color_aabbggrr: str):
    outer = [(float(x), float(y)) for x, y in list(poly.exterior.coords)]
    inners = []
    try:
        for ring in poly.interiors:
            inners.append([(float(x), float(y)) for x, y in list(ring.coords)])
    except Exception:
        inners = []

    if inners:
        pg = kml_obj.newpolygon(name=name, outerboundaryis=outer, innerboundaryis=inners)
    else:
        pg = kml_obj.newpolygon(name=name, outerboundaryis=outer)

    pg.style.polystyle.color = color_aabbggrr
    pg.style.polystyle.fill = 1
    pg.style.polystyle.outline = 1
    return pg

# ---------- utf_ken_all の用意（zipがあれば展開） ----------
if (not os.path.exists("utf_ken_all.csv")) and os.path.exists("utf_ken_all.zip"):
    with zipfile.ZipFile("utf_ken_all.zip") as z:
        z.extractall(".")
    print("utf_ken_all.zip を展開しました")

if not os.path.exists("utf_ken_all.csv"):
    raise FileNotFoundError("utf_ken_all.csv（または utf_ken_all.zip）が見つかりません。")

# ---------- 位置情報マスタ（utf_ken_all + ○○位置.csv） ----------
utf = read_csv_auto("utf_ken_all.csv", header_opt=None)
if utf.shape[1] < 9:
    raise ValueError("utf_ken_all.csv の列数が想定外です（KEN_ALL形式か確認してください）。")

utf.columns = [
    "jis", "old_zip", "zip7",
    "pref_kana", "city_kana", "town_kana",
    "pref", "city", "town"
] + [f"extra_{i}" for i in range(utf.shape[1] - 9)]

utf["zip7"] = utf["zip7"].astype(str).str.zfill(7)
utf_use = utf[["zip7", "pref", "city", "town"]].copy()
for c in ["pref", "city", "town"]:
    utf_use[c] = utf_use[c].astype(str).str.strip()
utf_use["town_base"] = utf_use["town"].map(make_town_base)

loc_list = []
for path in glob.glob("*位置*.csv"):
    print(f"位置参照ファイル候補: {path}")
    loc = read_csv_auto(path, header_opt=0)

    required = {"都道府県名", "市区町村名", "大字町丁目名", "緯度", "経度"}
    if not required.issubset(loc.columns):
        print(f"  {path} は必須列が足りないのでスキップ: {loc.columns}")
        continue

    df_pos = loc.rename(columns={
        "都道府県名": "pref",
        "市区町村名": "city",
        "大字町丁目名": "town",
        "緯度": "lat",
        "経度": "lon",
    }).copy()

    for c in ["pref", "city", "town"]:
        df_pos[c] = df_pos[c].astype(str).str.strip()

    df_pos["town_base"] = df_pos["town"].map(make_town_base)
    loc_list.append(df_pos[["pref", "city", "town_base", "lat", "lon"]])

if not loc_list:
    raise ValueError("『○○位置.csv』が見つからないか、必須列（都道府県名/市区町村名/大字町丁目名/緯度/経度）がありません。")

loc_all = pd.concat(loc_list, ignore_index=True).dropna(subset=["lat", "lon"])
loc_base = (
    loc_all
    .groupby(["pref", "city", "town_base"], as_index=False)
    .agg(lat=("lat", "mean"), lon=("lon", "mean"))
)

# ---------- 会場座標（直指定） ----------
venue_lat = float(VENUE_LAT)
venue_lon = float(VENUE_LON)
print(f"会場座標: name={VENUE_NAME}, lat={venue_lat}, lon={venue_lon}")

# ---------- 各回：2σカット点集合 + 面 ----------
all_event_keep_list = []
event_polygons = []   # (event_id, [poly1, poly2, ...])

for idx, csv_path in enumerate(VISITOR_CSV_LIST, start=1):
    print(f"\n=== イベント{idx}: {csv_path} ===")
    vis = read_csv_auto(csv_path, header_opt=None)
    vis.columns = [f"col_{i}" for i in range(vis.shape[1])]

    vis["zip7"] = (
        vis["col_0"].astype(str)
        .str.replace(r"\D", "", regex=True)
        .str.zfill(7)
    )
    vis = vis[vis["zip7"].str.match(r"^\d{7}$")].copy()
    if vis.empty:
        print("  有効郵便番号なし → スキップ")
        continue

    vis_addr = vis.merge(utf_use, on="zip7", how="left")
    if vis_addr[["pref", "city", "town"]].isna().all(axis=None):
        print("  utf_ken_all にマッチなし → スキップ")
        continue

    vis_addr["town_base"] = vis_addr["town"].map(make_town_base)
    vis_geo = (
        vis_addr.merge(loc_base, on=["pref", "city", "town_base"], how="left")
        .dropna(subset=["lat", "lon"])
        .copy()
    )
    if vis_geo.empty:
        print("  緯度経度が付かない → スキップ")
        continue

    vis_geo["lat"] = vis_geo["lat"].astype(float)
    vis_geo["lon"] = vis_geo["lon"].astype(float)

    # 距離（m）
    vis_geo["dist_m"] = [
        haversine_m(venue_lon, venue_lat, lo, la)
        for lo, la in zip(vis_geo["lon"], vis_geo["lat"])
    ]

    n0 = len(vis_geo)
    kept, mu, sd, thr = sigma_cut(vis_geo, dist_col="dist_m", k=2.0)
    n1 = len(kept)
    print(f"  レコード数 {n0} → 2σ以内 {n1}（しきい値={thr/1000:.2f}km, 平均={mu/1000:.2f}km, σ={sd/1000:.2f}km）")

    if kept.empty:
        print("  2σ後に0件 → スキップ")
        continue

    kept["event_id"] = idx

    coords = list(zip(kept["lon"], kept["lat"]))
    print("  concave hull 計算中（イベントごと）...")
    hull = build_alpha_shape(coords, target_ratio=0.1)

    # ★複数面化（上位10面、最大面積の2%未満は捨てる）
    polys = hull_to_polygons(hull, max_parts=10, min_area_ratio=0.02)
    print(f"  取得ポリゴン数: {len(polys)}")

    event_polygons.append((idx, polys))
    all_event_keep_list.append(kept)

if not all_event_keep_list:
    raise ValueError("有効なイベントデータが1件もありません。VISITOR_CSV_LIST とファイル内容を確認してください。")

# ---------- n回分を統合 → 『合計1回しか出ない郵便番号』を除外 ----------
all_keep = pd.concat(all_event_keep_list, ignore_index=True)

zip_counts = all_keep.groupby("zip7")["zip7"].transform("size")
mask_multi = zip_counts >= 2
removed_single = int((~mask_multi).sum())
core_base = all_keep[mask_multi].copy()

print(f"\n統合（各回2σ後）レコード総数: {len(all_keep)}")
print(f"『n回合計で1レコードしかない郵便番号』除外数: {removed_single}")
print(f"残りレコード数: {len(core_base)}")

if core_base.empty:
    raise ValueError("1回しか出てこない郵便番号を除いた結果、データが空になりました。")

# ---------- 統合データを 2σカット ----------
core_final, mu2, sd2, thr2 = sigma_cut(core_base, dist_col="dist_m", k=2.0)
print(f"統合ベース {len(core_base)} → 統合2σ以内 {len(core_final)}（しきい値={thr2/1000:.2f}km, 平均={mu2/1000:.2f}km, σ={sd2/1000:.2f}km）")

if core_final.empty:
    raise ValueError("統合2σカット後にデータが0件になりました。")

core_coords = list(zip(core_final["lon"], core_final["lat"]))
print("統合コア商圏 concave hull 計算中...")
core_hull = build_alpha_shape(core_coords, target_ratio=0.1)

# ★統合も複数面化
core_polys = hull_to_polygons(core_hull, max_parts=10, min_area_ratio=0.02)
print(f"統合コア：取得ポリゴン数 {len(core_polys)}")

# ---------- KML：面 ----------
kml_poly = simplekml.Kml()

# 会場ピン（面KML）
venue_pt_poly = kml_poly.newpoint(name=VENUE_NAME, coords=[(venue_lon, venue_lat)])
venue_pt_poly.style.iconstyle.scale = 1.2
venue_pt_poly.style.iconstyle.color = simplekml.Color.red

# 各回の面（薄青）：複数
for idx, polys in event_polygons:
    for j, pg in enumerate(polys, start=1):
        add_polygon(
            kml_poly,
            pg,
            name=f"event_{idx}_2sigma_area_{j}",
            color_aabbggrr=simplekml.Color.changealpha('3f', simplekml.Color.blue)
        )

# 統合コア面（濃紫）：複数
for j, pg in enumerate(core_polys, start=1):
    add_polygon(
        kml_poly,
        pg,
        name=f"core_merged_2sigma_area_{j}",
        color_aabbggrr=simplekml.Color.changealpha('7f', simplekml.Color.purple)
    )

kml_poly.save(OUTPUT_KML_POLY)
print(f"\n面KML 出力完了：{OUTPUT_KML_POLY}")

# ---------- KML：点 ----------
kml_pts = simplekml.Kml()

# 会場ピン（点KML）
venue_pt_pts = kml_pts.newpoint(name=VENUE_NAME, coords=[(venue_lon, venue_lat)])
venue_pt_pts.style.iconstyle.scale = 1.2
venue_pt_pts.style.iconstyle.color = simplekml.Color.red

# 各回の点
for i, kept in enumerate(all_event_keep_list, start=1):
    folder = kml_pts.newfolder(name=f"event_{i}_2sigma_points")
    for _, row in kept.iterrows():
        p = folder.newpoint(coords=[(float(row["lon"]), float(row["lat"]))])
        p.name = str(row.get("zip7", ""))
        p.style.iconstyle.scale = 0.5
        p.style.iconstyle.color = simplekml.Color.white

# 統合コア点（黄）
folder_core = kml_pts.newfolder(name="core_merged_2sigma_points")
for _, row in core_final.iterrows():
    p = folder_core.newpoint(coords=[(float(row["lon"]), float(row["lat"]))])
    p.name = str(row.get("zip7", ""))
    p.style.iconstyle.scale = 0.7
    p.style.iconstyle.color = simplekml.Color.yellow

kml_pts.save(OUTPUT_KML_POINTS)
print(f"点KML 出力完了：{OUTPUT_KML_POINTS}")

print("\n完了：")
print("  ・各回（2σ）面 + 統合コア面（複数面対応） →", OUTPUT_KML_POLY)
print("  ・各回（2σ）点 + 統合コア点 →", OUTPUT_KML_POINTS)


utf_ken_all.csv を encoding='utf-8-sig' で読み込み成功
位置参照ファイル候補: 愛媛位置.csv
愛媛位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 兵庫位置.csv
兵庫位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 青森位置.csv
青森位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 福井位置.csv
福井位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 大阪位置.csv
大阪位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 福岡位置.csv
福岡位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 福島位置.csv
福島位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 奈良位置.csv
奈良位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 滋賀位置.csv
滋賀位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 千葉位置.csv
千葉位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 茨城位置.csv
茨城位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 埼玉位置.csv
埼玉位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 長野位置.csv
長野位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 北海道位置.csv
北海道位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 宮崎位置.csv
宮崎位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 熊本位置.csv
熊本位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 大分位置.csv
大分位置.csv を encodin

In [15]:
# ===== 必要ライブラリインストール　一回のみ開催会場用 =====
!pip -q install simplekml shapely alphashape

# ===== 設定（ここだけ書き換え） =====
VISITOR_CSV = "AJ-42-9 1w-おのだサンパーク-新規対象来場332／760.csv"

# 会場は緯度経度で指定
VENUE_LAT  = 33.98526816958754
VENUE_LON  = 131.17094913818082
VENUE_NAME = "おのだサンパーク"

# 残す割合（0.60=60%, 0.80=80%など）
KEEP_RATIO = 0.70

# ★面の締まり具合（ここが肝）
CLOSE_KM  = 30.0   # 例: 8〜20あたりで調整（小さいほどタイト・大きいほど繋がる）
SMOOTH_KM = 2.0    # 0で無効。角を少し丸める（1〜5くらい）

OUTPUT_KML = f"venue_{int(KEEP_RATIO*100)}pct_latlon_closing.kml"

# ===== ここから下は触らなくてOK =====
import pandas as pd
import math, re, glob, os, zipfile
import numpy as np
import simplekml
from shapely.geometry import Point, MultiPoint
from shapely.ops import unary_union

def read_csv_auto(path, header_opt):
    encodings = ["cp932", "utf-8-sig", "utf-8"]
    last_err = None
    for enc in encodings:
        try:
            df = pd.read_csv(path, encoding=enc, header=header_opt)
            print(f"{path} を encoding='{enc}' で読み込み成功")
            return df
        except UnicodeDecodeError as e:
            last_err = e
            continue
        except FileNotFoundError as e:
            raise e
    raise UnicodeDecodeError(f"{path}", b"", 0, 1, f"試したエンコーディング {encodings} すべてで失敗: {last_err}")

def make_town_base(t):
    t = str(t)
    t = re.sub(r"（.*?）", "", t)
    t = re.sub(r"([一二三四五六七八九十〇零0-9]+)丁目$", "", t)
    return t.strip()

def project_m(lon, lat, lon0, lat0):
    # equirectangular（会場中心のローカル平面に変換）
    R = 6371000.0
    x = (math.radians(lon - lon0)) * R * math.cos(math.radians(lat0))
    y = (math.radians(lat - lat0)) * R
    return x, y

def unproject_deg(x, y, lon0, lat0):
    R = 6371000.0
    lon = lon0 + math.degrees(x / (R * math.cos(math.radians(lat0))))
    lat = lat0 + math.degrees(y / R)
    return lon, lat

def pick_one_polygon(geom, center_pt):
    # Polygon/MultiPolygon/GeometryCollectionから「中心に一番近い」面を1つ選ぶ
    if geom is None or geom.is_empty:
        return None
    g = geom
    if g.geom_type == "Polygon":
        return g
    if g.geom_type == "MultiPolygon":
        polys = list(g.geoms)
        return min(polys, key=lambda p: p.distance(center_pt))
    if hasattr(g, "geoms"):
        polys = [x for x in g.geoms if x.geom_type == "Polygon" and (not x.is_empty)]
        if not polys:
            return None
        return min(polys, key=lambda p: p.distance(center_pt))
    return None

# utf_ken_all.zip があれば展開
if (not os.path.exists("utf_ken_all.csv")) and os.path.exists("utf_ken_all.zip"):
    with zipfile.ZipFile("utf_ken_all.zip") as z:
        z.extractall(".")
    print("utf_ken_all.zip を展開しました")
if not os.path.exists("utf_ken_all.csv"):
    raise FileNotFoundError("utf_ken_all.csv（または utf_ken_all.zip）が見つかりません。")

# 1) 来場者データ（A列=郵便番号）
vis = read_csv_auto(VISITOR_CSV, header_opt=None)
vis.columns = [f"col_{i}" for i in range(vis.shape[1])]
vis["zip7"] = vis["col_0"].astype(str).str.replace(r"\D", "", regex=True).str.zfill(7)
vis = vis[vis["zip7"].str.match(r"^\d{7}$")].copy()
if vis.empty:
    raise ValueError("来場データに有効な郵便番号がありません（A列を確認してください）。")

# 2) utf_ken_all
utf = read_csv_auto("utf_ken_all.csv", header_opt=None)
if utf.shape[1] < 9:
    raise ValueError("utf_ken_all.csv の列数が想定外です（KEN_ALL形式か確認してください）。")
utf.columns = [
    "jis", "old_zip", "zip7",
    "pref_kana", "city_kana", "town_kana",
    "pref", "city", "town"
] + [f"extra_{i}" for i in range(utf.shape[1] - 9)]
utf["zip7"] = utf["zip7"].astype(str).str.zfill(7)
utf_use = utf[["zip7", "pref", "city", "town"]].copy()
for c in ["pref", "city", "town"]:
    utf_use[c] = utf_use[c].astype(str).str.strip()
utf_use["town_base"] = utf_use["town"].map(make_town_base)

# 3) 住所付与
vis_addr = vis.merge(utf_use, on="zip7", how="left")
if vis_addr[["pref", "city", "town"]].isna().all(axis=None):
    raise ValueError("来場データの郵便番号が utf_ken_all.csv とマッチしていません。")

# 4) *_2024.csv（位置参照）
loc_list = []
for path in glob.glob("*位置.csv"):
    print(f"位置参照ファイル候補: {path}")
    loc = read_csv_auto(path, header_opt=0)

    cols = {c: c for c in loc.columns}
    for k in list(cols):
        if "都道府県名" in k: cols[k] = "pref"
        if "市区町村名" in k: cols[k] = "city"
        if "大字町丁目名" in k: cols[k] = "town"
        if "緯度" in k: cols[k] = "lat"
        if "経度" in k: cols[k] = "lon"
    loc = loc.rename(columns=cols)

    needed = {"pref", "city", "town", "lat", "lon"}
    if not needed.issubset(loc.columns):
        print(f"{path} は必要列が揃っていないのでスキップ ({loc.columns})")
        continue

    for c in ["pref", "city", "town"]:
        loc[c] = loc[c].astype(str).str.strip()
    loc["town_base"] = loc["town"].map(make_town_base)
    loc_list.append(loc[["pref", "city", "town_base", "lat", "lon"]])

if not loc_list:
    raise ValueError("位置参照情報 *_2024.csv が見つからないか、必要列がありません。")

loc_all = pd.concat(loc_list, ignore_index=True).dropna(subset=["lat","lon"])
loc_base = (
    loc_all.groupby(["pref", "city", "town_base"], as_index=False)
    .agg(lat=("lat","mean"), lon=("lon","mean"))
)

# 5) 来場データに緯度経度
vis_addr["town_base"] = vis_addr["town"].map(make_town_base)
vis_geo = vis_addr.merge(loc_base, on=["pref","city","town_base"], how="left").dropna(subset=["lat","lon"]).copy()
if vis_geo.empty:
    raise ValueError("来場データに緯度経度が1件も付きませんでした。")

vis_geo["lat"] = vis_geo["lat"].astype(float)
vis_geo["lon"] = vis_geo["lon"].astype(float)

# 6) 会場（直指定）
venue_lat = float(VENUE_LAT)
venue_lon = float(VENUE_LON)
venue_point_deg = Point(venue_lon, venue_lat)
print(f"会場座標: {VENUE_NAME} lat={venue_lat}, lon={venue_lon}")

# 7) 近い順に KEEP_RATIO 残す
if not (0 < KEEP_RATIO <= 1.0):
    raise ValueError("KEEP_RATIO は 0 より大きく 1 以下にしてください。")
vis_geo["geom"] = [Point(lo, la) for lo, la in zip(vis_geo["lon"], vis_geo["lat"])]
vis_geo["dist"] = vis_geo["geom"].apply(lambda p: p.distance(venue_point_deg))  # 度（順位用）
vis_geo_sorted = vis_geo.sort_values("dist").reset_index(drop=True)
n = len(vis_geo_sorted)
keep_n = max(3, math.ceil(n * KEEP_RATIO))
vis_keep = vis_geo_sorted.iloc[:keep_n].copy()
print(f"点の採用: {keep_n}/{n}（{int(KEEP_RATIO*100)}%）")

# 8) ★closingで「1枚で、凸包ほど雑じゃない」面を作る（ローカル平面[m]で処理）
buf_m = float(CLOSE_KM) * 1000.0
sm_m  = float(SMOOTH_KM) * 1000.0

pts_m = [Point(*project_m(lo, la, venue_lon, venue_lat)) for lo, la in zip(vis_keep["lon"], vis_keep["lat"])]
mp_m = MultiPoint(pts_m)

# closing: buffer(+buf) -> buffer(-buf)
g = mp_m.buffer(buf_m)
g = g.buffer(-buf_m)

# smoothing（任意）
if sm_m > 0:
    g = g.buffer(sm_m).buffer(-sm_m)

# 念のため dissolve
g = unary_union(g)

center_m = Point(0.0, 0.0)
poly_m = pick_one_polygon(g, center_m)
if poly_m is None or poly_m.is_empty:
    raise ValueError("面が作れませんでした。CLOSE_KMを少し上げるか、KEEP_RATIOを上げてください。")

# 9) KML出力
kml = simplekml.Kml()

# 面（m→lon/latに戻す）
ext = list(poly_m.exterior.coords)
poly_coords = [unproject_deg(x, y, venue_lon, venue_lat) for x, y in ext]  # (lon,lat)
poly = kml.newpolygon(
    name=f"{int(KEEP_RATIO*100)}percent_area",
    outerboundaryis=[(lon, lat) for lon, lat in poly_coords],
)
poly.style.polystyle.color = simplekml.Color.changealpha('7f', simplekml.Color.gray)
poly.style.polystyle.fill = 1
poly.style.polystyle.outline = 1

# 会場ピン
venue_pt = kml.newpoint(name=VENUE_NAME, coords=[(venue_lon, venue_lat)])
venue_pt.style.iconstyle.scale = 1.2
venue_pt.style.iconstyle.color = simplekml.Color.red

# 点
folder_pts = kml.newfolder(name=f"{int(KEEP_RATIO*100)}percent_points")
for _, row in vis_keep.iterrows():
    p = folder_pts.newpoint(coords=[(row["lon"], row["lat"])])
    p.name = str(row.get("zip7",""))
    p.style.iconstyle.scale = 0.7
    p.style.iconstyle.color = simplekml.Color.white

kml.save(OUTPUT_KML)
print(f"完了：'{OUTPUT_KML}'（CLOSE_KM={CLOSE_KM}, SMOOTH_KM={SMOOTH_KM}）")


AJ-42-9 1w-おのだサンパーク-新規対象来場332／760.csv を encoding='cp932' で読み込み成功
utf_ken_all.csv を encoding='utf-8-sig' で読み込み成功
位置参照ファイル候補: 愛媛位置.csv
愛媛位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 兵庫位置.csv
兵庫位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 青森位置.csv
青森位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 福井位置.csv
福井位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 大阪位置.csv
大阪位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 福岡位置.csv
福岡位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 福島位置.csv
福島位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 奈良位置.csv
奈良位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 滋賀位置.csv
滋賀位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 千葉位置.csv
千葉位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 茨城位置.csv
茨城位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 埼玉位置.csv
埼玉位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 長野位置.csv
長野位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 北海道位置.csv
北海道位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 宮崎位置.csv
宮崎位置.csv を encoding='cp932' で読み込み成功
位置参照ファイル候補: 熊本位置.csv
熊本位置.csv を

In [ ]:
# ===============================================
# 郵便番号マッピング → 複数回開催 n 回分の統合コア商圏（○○位置.csv 版）
# - 会場座標は郵便番号ではなく「緯度経度を直指定」
# - 2σカット（平均+2σ 以内を残す）
# - 出力を「点KML」「面KML」に分割
# ===============================================

!pip -q install simplekml shapely alphashape

# ===== 設定（ここだけ自分の環境に合わせて書き換え） =====
VISITOR_CSV_LIST = [
    "AJ-41-5-4w-青森東奥日報新町ビル-新規対象来場104／255.csv",
    "AJ-42-5 3w-東奥日報新町ビル-新規対象来場201／524.csv"
]

# ★会場座標を直指定（緯度, 経度）
VENUE_LAT = 40.82574171229414# 例：サッポロファクトリー近辺（適宜あなたの値に）
VENUE_LON = 140.74092972351494# 例：サッポロファクトリー近辺（適宜あなたの値に）
VENUE_NAME = "青森東奥日報新町ビル"  # KML表示名（任意）

# 出力（面KML / 点KML）
OUTPUT_KML_POLY   = "青森東奥日報集客エリア.kml"
OUTPUT_KML_POINTS = "青森産業会館集客エリア点.kml"

# ===== ここから下は触らなくてOK =====
import pandas as pd
import math, re, glob, os, zipfile
import numpy as np
import simplekml
import alphashape
from shapely.geometry import Point, MultiPoint

# ---------- 共通関数 ----------
def read_csv_auto(path, header_opt):
    encodings = ["cp932", "utf-8-sig", "utf-8"]
    last_err = None
    for enc in encodings:
        try:
            df = pd.read_csv(path, encoding=enc, header=header_opt)
            print(f"{path} を encoding='{enc}' で読み込み成功")
            return df
        except UnicodeDecodeError as e:
            last_err = e
            continue
        except FileNotFoundError as e:
            raise e
    raise UnicodeDecodeError(f"{path}", b"", 0, 1, f"試したエンコーディング {encodings} すべてで失敗: {last_err}")

def make_town_base(t):
    t = str(t)
    t = re.sub(r"（.*?）", "", t)
    t = re.sub(r"([一二三四五六七八九十〇零0-9]+)丁目$", "", t)
    return t.strip()

def haversine_m(lon1, lat1, lon2, lat2):
    R = 6371000.0
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlmb = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlmb/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c

def sigma_cut(df, dist_col="dist_m", k=2.0):
    if df.empty:
        return df, None, None, None
    mu = float(df[dist_col].mean())
    sd = float(df[dist_col].std(ddof=0))
    thr = mu + k * sd
    out = df[df[dist_col] <= thr].copy()
    return out, mu, sd, thr

def build_alpha_shape(coords, target_ratio=0.1):
    if len(coords) < 3:
        return None

    mp_all = MultiPoint(coords)
    convex = mp_all.convex_hull
    convex_area = convex.area

    if convex_area == 0:
        poly = convex
    else:
        alpha_candidates = np.logspace(-2.3, 1, 24)
        best_alpha = None
        best_shape = None
        best_diff = float("inf")

        for a in alpha_candidates:
            try:
                shp = alphashape.alphashape(coords, a)
            except Exception:
                continue
            if shp.is_empty or getattr(shp, "area", 0) == 0:
                continue

            ratio = shp.area / convex_area
            diff = abs(ratio - target_ratio)
            if diff < best_diff:
                best_diff = diff
                best_alpha = a
                best_shape = shp

        print("  chosen alpha:", best_alpha, " diff_from_target=", best_diff)
        poly = best_shape if best_shape is not None else convex

    if poly.geom_type in ("LineString", "MultiLineString", "Point", "MultiPoint"):
        poly = poly.buffer(1e-4)
    return poly

def hull_to_polygon(hull):
    if hull is None or hull.is_empty:
        return None
    if hull.geom_type == "MultiPolygon":
        hull = max(list(hull.geoms), key=lambda g: g.area)
    if hull.geom_type != "Polygon":
        hull = hull.buffer(1e-4)
    if hull.geom_type != "Polygon" or hull.is_empty:
        return None
    return hull

def add_polygon(kml_obj, hull_poly, name, color_aabbggrr):
    coords = list(hull_poly.exterior.coords)
    poly = kml_obj.newpolygon(
        name=name,
        outerboundaryis=[(lon, lat) for lon, lat in coords],
    )
    poly.style.polystyle.color = color_aabbggrr
    poly.style.polystyle.fill = 1
    poly.style.polystyle.outline = 1
    return poly

# ---------- utf_ken_all の用意（zipがあれば展開） ----------
if (not os.path.exists("utf_ken_all.csv")) and os.path.exists("utf_ken_all.zip"):
    with zipfile.ZipFile("utf_ken_all.zip") as z:
        z.extractall(".")
    print("utf_ken_all.zip を展開しました")

if not os.path.exists("utf_ken_all.csv"):
    raise FileNotFoundError("utf_ken_all.csv（または utf_ken_all.zip）が見つかりません。")

# ---------- 位置情報マスタ（utf_ken_all + ○○位置.csv） ----------
utf = read_csv_auto("utf_ken_all.csv", header_opt=None)
if utf.shape[1] < 9:
    raise ValueError("utf_ken_all.csv の列数が想定外です（KEN_ALL形式か確認してください）。")

utf.columns = [
    "jis", "old_zip", "zip7",
    "pref_kana", "city_kana", "town_kana",
    "pref", "city", "town"
] + [f"extra_{i}" for i in range(utf.shape[1] - 9)]

utf["zip7"] = utf["zip7"].astype(str).str.zfill(7)
utf_use = utf[["zip7", "pref", "city", "town"]].copy()
for c in ["pref", "city", "town"]:
    utf_use[c] = utf_use[c].astype(str).str.strip()
utf_use["town_base"] = utf_use["town"].map(make_town_base)

loc_list = []
for path in glob.glob("*位置*.csv"):
    print(f"位置参照ファイル候補: {path}")
    loc = read_csv_auto(path, header_opt=0)
    required = {"都道府県名", "市区町村名", "大字町丁目名", "緯度", "経度"}
    if not required.issubset(loc.columns):
        print(f"  {path} は必須列が足りないのでスキップ: {loc.columns}")
        continue

    df_pos = loc.rename(columns={
        "都道府県名": "pref",
        "市区町村名": "city",
        "大字町丁目名": "town",
        "緯度": "lat",
        "経度": "lon",
    }).copy()

    for c in ["pref", "city", "town"]:
        df_pos[c] = df_pos[c].astype(str).str.strip()

    df_pos["town_base"] = df_pos["town"].map(make_town_base)
    loc_list.append(df_pos[["pref", "city", "town_base", "lat", "lon"]])

if not loc_list:
    raise ValueError("『○○位置.csv』が見つからないか、必須列（都道府県名/市区町村名/大字町丁目名/緯度/経度）がありません。")

loc_all = pd.concat(loc_list, ignore_index=True).dropna(subset=["lat", "lon"])
loc_base = (
    loc_all
    .groupby(["pref", "city", "town_base"], as_index=False)
    .agg(lat=("lat", "mean"), lon=("lon", "mean"))
)

# ---------- ★会場座標（直指定） ----------
venue_lat = float(VENUE_LAT)
venue_lon = float(VENUE_LON)
print(f"会場座標: name={VENUE_NAME}, lat={venue_lat}, lon={venue_lon}")

# ---------- 各回：2σカット点集合 + 面 ----------
all_event_keep_list = []
event_polygons = []

for idx, csv_path in enumerate(VISITOR_CSV_LIST, start=1):
    print(f"\n=== イベント{idx}: {csv_path} ===")
    vis = read_csv_auto(csv_path, header_opt=None)
    vis.columns = [f"col_{i}" for i in range(vis.shape[1])]

    vis["zip7"] = (
        vis["col_0"].astype(str)
        .str.replace(r"\D", "", regex=True)
        .str.zfill(7)
    )
    vis = vis[vis["zip7"].str.match(r"^\d{7}$")].copy()
    if vis.empty:
        print("  有効郵便番号なし → スキップ")
        continue

    vis_addr = vis.merge(utf_use, on="zip7", how="left")
    if vis_addr[["pref", "city", "town"]].isna().all(axis=None):
        print("  utf_ken_all にマッチなし → スキップ")
        continue

    vis_addr["town_base"] = vis_addr["town"].map(make_town_base)
    vis_geo = vis_addr.merge(loc_base, on=["pref", "city", "town_base"], how="left").dropna(subset=["lat", "lon"]).copy()
    if vis_geo.empty:
        print("  緯度経度が付かない → スキップ")
        continue

    vis_geo["lat"] = vis_geo["lat"].astype(float)
    vis_geo["lon"] = vis_geo["lon"].astype(float)

    # 距離（m）★会場座標に対して
    vis_geo["dist_m"] = [
        haversine_m(venue_lon, venue_lat, lo, la)
        for lo, la in zip(vis_geo["lon"], vis_geo["lat"])
    ]

    n0 = len(vis_geo)
    kept, mu, sd, thr = sigma_cut(vis_geo, dist_col="dist_m", k=2.0)
    n1 = len(kept)
    print(f"  レコード数 {n0} → 2σ以内 {n1}（しきい値={thr/1000:.2f}km, 平均={mu/1000:.2f}km, σ={sd/1000:.2f}km）")

    if kept.empty:
        print("  2σ後に0件 → スキップ")
        continue

    kept["event_id"] = idx

    coords = list(zip(kept["lon"], kept["lat"]))
    print("  concave hull 計算中（イベントごと）...")
    hull = build_alpha_shape(coords, target_ratio=0.1)
    hull_poly = hull_to_polygon(hull)
    event_polygons.append((idx, hull_poly))

    all_event_keep_list.append(kept)

if not all_event_keep_list:
    raise ValueError("有効なイベントデータが1件もありません。VISITOR_CSV_LIST とファイル内容を確認してください。")

# ---------- n回分を統合 → 『合計1回しか出ない郵便番号』を除外 ----------
all_keep = pd.concat(all_event_keep_list, ignore_index=True)
zip_counts = all_keep.groupby("zip7")["zip7"].transform("size")
mask_multi = zip_counts >= 2
removed_single = int((~mask_multi).sum())
core_base = all_keep[mask_multi].copy()

print(f"\n統合（各回2σ後）レコード総数: {len(all_keep)}")
print(f"『n回合計で1レコードしかない郵便番号』除外数: {removed_single}")
print(f"残りレコード数: {len(core_base)}")

if core_base.empty:
    raise ValueError("1回しか出てこない郵便番号を除いた結果、データが空になりました。")

# ---------- 統合データを 2σカット ----------
core_final, mu2, sd2, thr2 = sigma_cut(core_base, dist_col="dist_m", k=2.0)
print(f"統合ベース {len(core_base)} → 統合2σ以内 {len(core_final)}（しきい値={thr2/1000:.2f}km, 平均={mu2/1000:.2f}km, σ={sd2/1000:.2f}km）")

if core_final.empty:
    raise ValueError("統合2σカット後にデータが0件になりました。")

core_coords = list(zip(core_final["lon"], core_final["lat"]))
print("統合コア商圏 concave hull 計算中...")
core_hull = build_alpha_shape(core_coords, target_ratio=0.1)
core_hull_poly = hull_to_polygon(core_hull)

# ---------- KML：面 ----------
kml_poly = simplekml.Kml()
venue_pt_poly = kml_poly.newpoint(name=VENUE_NAME, coords=[(venue_lon, venue_lat)])
venue_pt_poly.style.iconstyle.scale = 1.2
venue_pt_poly.style.iconstyle.color = simplekml.Color.red

for idx, hull_poly in event_polygons:
    if hull_poly is None:
        continue
    add_polygon(kml_poly, hull_poly, f"event_{idx}_2sigma_area",
                simplekml.Color.changealpha('3f', simplekml.Color.blue))

if core_hull_poly is not None:
    add_polygon(kml_poly, core_hull_poly, "core_merged_2sigma_area",
                simplekml.Color.changealpha('7f', simplekml.Color.purple))

kml_poly.save(OUTPUT_KML_POLY)
print(f"\n面KML 出力完了：{OUTPUT_KML_POLY}")

# ---------- KML：点 ----------
kml_pts = simplekml.Kml()
venue_pt_pts = kml_pts.newpoint(name=VENUE_NAME, coords=[(venue_lon, venue_lat)])
venue_pt_pts.style.iconstyle.scale = 1.2
venue_pt_pts.style.iconstyle.color = simplekml.Color.red

for i, kept in enumerate(all_event_keep_list, start=1):
    folder = kml_pts.newfolder(name=f"event_{i}_2sigma_points")
    for _, row in kept.iterrows():
        p = folder.newpoint(coords=[(float(row["lon"]), float(row["lat"]))])
        p.name = str(row.get("zip7", ""))
        p.style.iconstyle.scale = 0.5
        p.style.iconstyle.color = simplekml.Color.white

folder_core = kml_pts.newfolder(name="core_merged_2sigma_points")
for _, row in core_final.iterrows():
    p = folder_core.newpoint(coords=[(float(row["lon"]), float(row["lat"]))])
    p.name = str(row.get("zip7", ""))
    p.style.iconstyle.scale = 0.7
    p.style.iconstyle.color = simplekml.Color.yellow

kml_pts.save(OUTPUT_KML_POINTS)
print(f"点KML 出力完了：{OUTPUT_KML_POINTS}")

print("\n完了：")
print("  ・各回（2σ）面 + 統合コア面 →", OUTPUT_KML_POLY)
print("  ・各回（2σ）点 + 統合コア点 →", OUTPUT_KML_POINTS)


utf_ken_all.csv を encoding='utf-8-sig' で読み込み成功
位置参照ファイル候補: 青森位置.csv
青森位置.csv を encoding='cp932' で読み込み成功
会場座標: name=青森東奥日報新町ビル, lat=40.82574171229414, lon=140.74092972351494

=== イベント1: AJ-41-5-4w-青森東奥日報新町ビル-新規対象来場104／255.csv ===
AJ-41-5-4w-青森東奥日報新町ビル-新規対象来場104／255.csv を encoding='cp932' で読み込み成功
  レコード数 54 → 2σ以内 54（しきい値=94.10km, 平均=34.07km, σ=30.01km）
  concave hull 計算中（イベントごと）...
  chosen alpha: 7.186571307235264  diff_from_target= 0.04972013514295526

=== イベント2: AJ-42-5 3w-東奥日報新町ビル-新規対象来場201／524.csv ===
AJ-42-5 3w-東奥日報新町ビル-新規対象来場201／524.csv を encoding='cp932' で読み込み成功
  レコード数 79 → 2σ以内 79（しきい値=87.92km, 平均=29.02km, σ=29.45km）
  concave hull 計算中（イベントごと）...
  chosen alpha: 7.186571307235264  diff_from_target= 0.0038153152720446643

統合（各回2σ後）レコード総数: 133
『n回合計で1レコードしかない郵便番号』除外数: 61
残りレコード数: 72
統合ベース 72 → 統合2σ以内 72（しきい値=84.26km, 平均=24.42km, σ=29.92km）
統合コア商圏 concave hull 計算中...
  chosen alpha: 7.186571307235264  diff_from_target= 0.025377205700196775

面KML 出力完了：青森東奥日報集客エリア.kml
点KML 出力完了：青森産

In [ ]:
# ================ちぎれ対策
# 郵便番号マッピング → 複数回開催 n 回分の統合コア商圏（○○位置.csv 版）
# - 会場座標は「緯度経度を直指定」
# - 2σカット（平均+2σ 以内を残す）
# - 出力を「点KML」「面KML」に分割
# - core_merged_2sigma_area：MultiPolygonは“全部”出す
# - simplekml の innerboundaryis=None で落ちる件を修正（穴が無い時は引数を渡さない）
# ===============================================

!pip -q install simplekml shapely alphashape

# ===== 設定（ここだけ書き換え） =====
VISITOR_CSV_LIST = [
    "AJ-41-4-2w-釧路文化ホール-新規対象来場283／788.csv",
    "AJ-42-4 2w-コーチャンフォー釧路文化ホール(釧路市民文化会館)-新規対象来場177／484.csv",
]

# ★会場座標（緯度, 経度）を直指定
VENUE_LAT  = 42.9849
VENUE_LON  = 144.3816
VENUE_NAME = "Kushiro_BunkaHall"

SIGMA_K = 2.0
TARGET_RATIO = 0.1

OUTPUT_KML_POLY   = "釧路文化ホール_面_2sigma.kml"
OUTPUT_KML_POINTS = "釧路文化ホール_点_2sigma.kml"

# ===== ここから下は触らなくてOK =====
import pandas as pd
import math, re, glob, os, zipfile
import numpy as np
import simplekml
import alphashape
from shapely.geometry import MultiPoint

def read_csv_auto(path, header_opt):
    encodings = ["cp932", "utf-8-sig", "utf-8"]
    last_err = None
    for enc in encodings:
        try:
            df = pd.read_csv(path, encoding=enc, header=header_opt)
            print(f"{path} を encoding='{enc}' で読み込み成功")
            return df
        except UnicodeDecodeError as e:
            last_err = e
            continue
        except FileNotFoundError as e:
            raise e
    raise UnicodeDecodeError(f"{path}", b"", 0, 1, f"試したエンコーディング {encodings} すべてで失敗: {last_err}")

def make_town_base(t):
    t = str(t)
    t = re.sub(r"（.*?）", "", t)
    t = re.sub(r"([一二三四五六七八九十〇零0-9]+)丁目$", "", t)
    return t.strip()

def haversine_m(lon1, lat1, lon2, lat2):
    R = 6371000.0
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlmb = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlmb/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c

def sigma_cut(df, dist_col="dist_m", k=2.0):
    if df.empty:
        return df, None, None, None
    mu = float(df[dist_col].mean())
    sd = float(df[dist_col].std(ddof=0))
    thr = mu + k * sd
    out = df[df[dist_col] <= thr].copy()
    return out, mu, sd, thr

def build_alpha_shape(coords, target_ratio=0.1):
    if len(coords) < 3:
        return None

    mp_all = MultiPoint(coords)
    convex = mp_all.convex_hull
    convex_area = convex.area

    if convex_area == 0:
        poly = convex
    else:
        alpha_candidates = np.logspace(-2.3, 1, 24)
        best_alpha = None
        best_shape = None
        best_diff = float("inf")

        for a in alpha_candidates:
            try:
                shp = alphashape.alphashape(coords, a)
            except Exception:
                continue
            if shp is None or shp.is_empty:
                continue
            area = getattr(shp, "area", 0)
            if area == 0:
                continue

            ratio = area / convex_area
            diff = abs(ratio - target_ratio)
            if diff < best_diff:
                best_diff = diff
                best_alpha = a
                best_shape = shp

        print("  chosen alpha:", best_alpha, " diff_from_target=", best_diff)
        poly = best_shape if best_shape is not None else convex

    if poly is not None and poly.geom_type in ("LineString", "MultiLineString", "Point", "MultiPoint"):
        poly = poly.buffer(1e-4)

    return poly

def hull_to_polygons(hull):
    if hull is None:
        return []
    if getattr(hull, "is_empty", True):
        return []

    g = hull
    if g.geom_type in ("LineString", "MultiLineString", "Point", "MultiPoint"):
        g = g.buffer(1e-4)
        if g.is_empty:
            return []

    if g.geom_type == "Polygon":
        return [g]
    if g.geom_type == "MultiPolygon":
        return list(g.geoms)

    if hasattr(g, "geoms"):
        return [x for x in g.geoms if x.geom_type == "Polygon" and (not x.is_empty)]

    return []

def add_polygon(kml_obj, poly_geom, name, color_aabbggrr):
    """
    ★simplekml は innerboundaryis=None で落ちることがあるので、
      - 穴がある時だけ innerboundaryis を渡す
      - 無い時は引数を渡さない
    """
    outer = [(lon, lat) for lon, lat in list(poly_geom.exterior.coords)]
    inners = []
    for ring in poly_geom.interiors:
        inners.append([(lon, lat) for lon, lat in list(ring.coords)])

    if inners:
        poly = kml_obj.newpolygon(name=name, outerboundaryis=outer, innerboundaryis=inners)
    else:
        poly = kml_obj.newpolygon(name=name, outerboundaryis=outer)

    poly.style.polystyle.color = color_aabbggrr
    poly.style.polystyle.fill = 1
    poly.style.polystyle.outline = 1
    return poly

# ---------- utf_ken_all の用意 ----------
if (not os.path.exists("utf_ken_all.csv")) and os.path.exists("utf_ken_all.zip"):
    with zipfile.ZipFile("utf_ken_all.zip") as z:
        z.extractall(".")
    print("utf_ken_all.zip を展開しました")

if not os.path.exists("utf_ken_all.csv"):
    raise FileNotFoundError("utf_ken_all.csv（または utf_ken_all.zip）が見つかりません。")

# ---------- 位置情報マスタ ----------
utf = read_csv_auto("utf_ken_all.csv", header_opt=None)
if utf.shape[1] < 9:
    raise ValueError("utf_ken_all.csv の列数が想定外です（KEN_ALL形式か確認してください）。")

utf.columns = [
    "jis", "old_zip", "zip7",
    "pref_kana", "city_kana", "town_kana",
    "pref", "city", "town"
] + [f"extra_{i}" for i in range(utf.shape[1] - 9)]

utf["zip7"] = utf["zip7"].astype(str).str.zfill(7)
utf_use = utf[["zip7", "pref", "city", "town"]].copy()
for c in ["pref", "city", "town"]:
    utf_use[c] = utf_use[c].astype(str).str.strip()
utf_use["town_base"] = utf_use["town"].map(make_town_base)

loc_list = []
for path in glob.glob("*位置*.csv"):
    print(f"位置参照ファイル候補: {path}")
    loc = read_csv_auto(path, header_opt=0)
    required = {"都道府県名", "市区町村名", "大字町丁目名", "緯度", "経度"}
    if not required.issubset(loc.columns):
        print(f"  {path} は必須列が足りないのでスキップ: {loc.columns}")
        continue

    df_pos = loc.rename(columns={
        "都道府県名": "pref",
        "市区町村名": "city",
        "大字町丁目名": "town",
        "緯度": "lat",
        "経度": "lon",
    }).copy()

    for c in ["pref", "city", "town"]:
        df_pos[c] = df_pos[c].astype(str).str.strip()

    df_pos["town_base"] = df_pos["town"].map(make_town_base)
    loc_list.append(df_pos[["pref", "city", "town_base", "lat", "lon"]])

if not loc_list:
    raise ValueError("『○○位置.csv』が見つからないか、必須列（都道府県名/市区町村名/大字町丁目名/緯度/経度）がありません。")

loc_all = pd.concat(loc_list, ignore_index=True).dropna(subset=["lat", "lon"])
loc_base = (
    loc_all
    .groupby(["pref", "city", "town_base"], as_index=False)
    .agg(lat=("lat", "mean"), lon=("lon", "mean"))
)

# ---------- 会場座標 ----------
venue_lat = float(VENUE_LAT)
venue_lon = float(VENUE_LON)
print(f"\n会場座標: name={VENUE_NAME}, lat={venue_lat}, lon={venue_lon}")

# ---------- 各回：2σカット + hull ----------
all_event_keep_list = []
event_hulls = []  # (event_id, hull)

for idx, csv_path in enumerate(VISITOR_CSV_LIST, start=1):
    print(f"\n=== イベント{idx}: {csv_path} ===")
    vis = read_csv_auto(csv_path, header_opt=None)
    vis.columns = [f"col_{i}" for i in range(vis.shape[1])]

    vis["zip7"] = (
        vis["col_0"].astype(str)
        .str.replace(r"\D", "", regex=True)
        .str.zfill(7)
    )
    vis = vis[vis["zip7"].str.match(r"^\d{7}$")].copy()
    if vis.empty:
        print("  有効郵便番号なし → スキップ")
        continue

    vis_addr = vis.merge(utf_use, on="zip7", how="left")
    if vis_addr[["pref", "city", "town"]].isna().all(axis=None):
        print("  utf_ken_all にマッチなし → スキップ")
        continue

    vis_addr["town_base"] = vis_addr["town"].map(make_town_base)
    vis_geo = vis_addr.merge(loc_base, on=["pref", "city", "town_base"], how="left").dropna(subset=["lat", "lon"]).copy()
    if vis_geo.empty:
        print("  緯度経度が付かない → スキップ")
        continue

    vis_geo["lat"] = vis_geo["lat"].astype(float)
    vis_geo["lon"] = vis_geo["lon"].astype(float)

    vis_geo["dist_m"] = [
        haversine_m(venue_lon, venue_lat, lo, la)
        for lo, la in zip(vis_geo["lon"], vis_geo["lat"])
    ]

    n0 = len(vis_geo)
    kept, mu, sd, thr = sigma_cut(vis_geo, dist_col="dist_m", k=SIGMA_K)
    n1 = len(kept)
    print(f"  レコード数 {n0} → 2σ以内 {n1}（しきい値={thr/1000:.2f}km, 平均={mu/1000:.2f}km, σ={sd/1000:.2f}km）")

    if kept.empty:
        print("  2σ後に0件 → スキップ")
        continue

    kept["event_id"] = idx

    coords = list(zip(kept["lon"], kept["lat"]))
    print("  concave hull 計算中（イベントごと）...")
    hull = build_alpha_shape(coords, target_ratio=TARGET_RATIO)

    all_event_keep_list.append(kept)
    event_hulls.append((idx, hull))

if not all_event_keep_list:
    raise ValueError("有効なイベントデータが1件もありません。VISITOR_CSV_LIST とファイル内容を確認してください。")

# ---------- 統合 → 合計1回しか出ない郵便番号を除外 ----------
all_keep = pd.concat(all_event_keep_list, ignore_index=True)
zip_counts = all_keep.groupby("zip7")["zip7"].transform("size")
mask_multi = zip_counts >= 2
removed_single = int((~mask_multi).sum())
core_base = all_keep[mask_multi].copy()

print(f"\n統合（各回2σ後）レコード総数: {len(all_keep)}")
print(f"『n回合計で1レコードしかない郵便番号』除外数: {removed_single}")
print(f"残りレコード数: {len(core_base)}")

if core_base.empty:
    raise ValueError("1回しか出てこない郵便番号を除いた結果、データが空になりました。")

# ---------- 統合をさらに2σカット ----------
core_final, mu2, sd2, thr2 = sigma_cut(core_base, dist_col="dist_m", k=SIGMA_K)
print(f"統合ベース {len(core_base)} → 統合2σ以内 {len(core_final)}（しきい値={thr2/1000:.2f}km, 平均={mu2/1000:.2f}km, σ={sd2/1000:.2f}km）")

if core_final.empty:
    raise ValueError("統合2σカット後にデータが0件になりました。")

core_coords = list(zip(core_final["lon"], core_final["lat"]))
print("統合コア商圏 concave hull 計算中...")
core_hull = build_alpha_shape(core_coords, target_ratio=TARGET_RATIO)

# ---------- 面KML ----------
kml_poly = simplekml.Kml()
venue_pt_poly = kml_poly.newpoint(name=VENUE_NAME, coords=[(venue_lon, venue_lat)])
venue_pt_poly.style.iconstyle.scale = 1.2
venue_pt_poly.style.iconstyle.color = simplekml.Color.red

# 各回面（薄青）※MultiPolygonなら全部出す
for idx, hull in event_hulls:
    polys = hull_to_polygons(hull)
    for j, pg in enumerate(polys, start=1):
        add_polygon(kml_poly, pg, f"event_{idx}_2sigma_area_{j}",
                    simplekml.Color.changealpha('3f', simplekml.Color.blue))

# 統合コア面（濃紫）※MultiPolygonなら全部出す
core_polys = hull_to_polygons(core_hull)
for j, pg in enumerate(core_polys, start=1):
    add_polygon(kml_poly, pg, f"core_merged_2sigma_area_{j}",
                simplekml.Color.changealpha('7f', simplekml.Color.purple))

kml_poly.save(OUTPUT_KML_POLY)
print(f"\n面KML 出力完了：{OUTPUT_KML_POLY}")

# ---------- 点KML ----------
kml_pts = simplekml.Kml()
venue_pt_pts = kml_pts.newpoint(name=VENUE_NAME, coords=[(venue_lon, venue_lat)])
venue_pt_pts.style.iconstyle.scale = 1.2
venue_pt_pts.style.iconstyle.color = simplekml.Color.red

# 各回点（白）
for i, kept in enumerate(all_event_keep_list, start=1):
    folder = kml_pts.newfolder(name=f"event_{i}_2sigma_points")
    for _, row in kept.iterrows():
        p = folder.newpoint(coords=[(float(row["lon"]), float(row["lat"]))])
        p.name = str(row.get("zip7", ""))
        p.style.iconstyle.scale = 0.5
        p.style.iconstyle.color = simplekml.Color.white

# 統合点（黄）
folder_core = kml_pts.newfolder(name="core_merged_2sigma_points")
for _, row in core_final.iterrows():
    p = folder_core.newpoint(coords=[(float(row["lon"]), float(row["lat"]))])
    p.name = str(row.get("zip7", ""))
    p.style.iconstyle.scale = 0.7
    p.style.iconstyle.color = simplekml.Color.yellow

kml_pts.save(OUTPUT_KML_POINTS)
print(f"点KML 出力完了：{OUTPUT_KML_POINTS}")

print("\n完了：")
print("  ・面KML →", OUTPUT_KML_POLY)
print("  ・点KML →", OUTPUT_KML_POINTS)


utf_ken_all.csv を encoding='utf-8-sig' で読み込み成功
位置参照ファイル候補: 北海道位置.csv
北海道位置.csv を encoding='cp932' で読み込み成功

会場座標: name=Kushiro_BunkaHall, lat=42.9849, lon=144.3816

=== イベント1: AJ-41-4-2w-釧路文化ホール-新規対象来場283／788.csv ===
AJ-41-4-2w-釧路文化ホール-新規対象来場283／788.csv を encoding='cp932' で読み込み成功
  レコード数 255 → 2σ以内 231（しきい値=88.07km, 平均=18.45km, σ=34.81km）
  concave hull 計算中（イベントごと）...
  chosen alpha: 7.186571307235264  diff_from_target= 0.002122476649111832

=== イベント2: AJ-42-4 2w-コーチャンフォー釧路文化ホール(釧路市民文化会館)-新規対象来場177／484.csv ===
AJ-42-4 2w-コーチャンフォー釧路文化ホール(釧路市民文化会館)-新規対象来場177／484.csv を encoding='cp932' で読み込み成功
  レコード数 167 → 2σ以内 162（しきい値=136.46km, 平均=34.65km, σ=50.90km）
  concave hull 計算中（イベントごと）...
  chosen alpha: 5.164680715397716  diff_from_target= 0.019806096825445443

統合（各回2σ後）レコード総数: 393
『n回合計で1レコードしかない郵便番号』除外数: 70
残りレコード数: 323
統合ベース 323 → 統合2σ以内 302（しきい値=54.52km, 平均=10.44km, σ=22.04km）
統合コア商圏 concave hull 計算中...
  chosen alpha: 10.0  diff_from_target= 0.02700003824084407

面KML 出力完了：釧路文化ホール_面_2sigma.

In [ ]:
# ===============================================
# 郵便番号マッピング → 複数回開催 n 回分の統合コア商圏（○○位置.csv 版）
# -----------------------------------------------
# すること：
#  1) 各回ごとに「会場からの距離で近い順90％」の点集合を作る
#  2) n回分の90％点を全部まとめて：
#      - 郵便番号ごとの合計レコード数をカウント
#      - 「合計1レコードしかない郵便番号」を除外
#      - 残りを重複込みで数え直し、会場から遠い10％をカット
#        → 残り90％で concave hull を作り「統合コア商圏」にする
#  3) 各回の90％面＋統合コア面＋会場ピン＋点群を1つのKMLに出力
#
# 必要ファイル（Colab にアップロードしてから実行）：
#   - utf_ken_all.csv
#   - ○○位置.csv （例: 福岡位置.csv, 愛知位置.csv ... 複数アップロード可）
#       列: 「都道府県名」「市区町村名」「大字町丁目名」「緯度」「経度」
#   - 過去 n 回ぶんの来場CSV : A列に郵便番号（B列以降はそのまま持つ）
# ===============================================

!pip install -q simplekml shapely alphashape

# ===== 設定（ここだけ自分の環境に合わせて書き換え） =====
VISITOR_CSV_LIST = [
    "AJ-41-10 4w-サッポロファクトリーホール-新規対象来場393／1255.csv",
    "AJ-41-4-1w-サッポロファクトリー-新規対象来場413／1177.csv",
    "AJ-41-7-3w-サッポロファクトリーホール-新規対象来場480／1421.csv",
     "AJ-42-4 3w-サッポロファクトリーホール-新規対象来場286／985.csv",
     "AJ-42-7 3w-サッポロファクトリーホール-神絵祭-華-新規対象来場575／2043.csv"
    # ...
]
VENUE_ZIP   = "060-0032"          # 会場の郵便番号（ハイフンあり/なしどちらでもOK）
OUTPUT_KML  = "サッポロファクトリーホール5回_90pct.kml"  # 出力KMLファイル名

# ===== ここから下は触らなくてOK =====
import pandas as pd
import math, re, glob
import numpy as np
import simplekml
import alphashape
from shapely.geometry import Point, MultiPoint

# ---------- 共通関数 ----------

def read_csv_auto(path, header_opt):
    """
    cp932 → utf-8-sig → utf-8 の順で読み込みを試す簡易関数
    header_opt: None -> header=None, 0 -> header=0
    """
    encodings = ["cp932", "utf-8-sig", "utf-8"]
    last_err = None
    for enc in encodings:
        try:
            df = pd.read_csv(path, encoding=enc, header=header_opt)
            print(f"{path} を encoding='{enc}' で読み込み成功")
            return df
        except UnicodeDecodeError as e:
            last_err = e
            continue
        except FileNotFoundError as e:
            raise e
    raise UnicodeDecodeError(f"{path}", b"", 0, 1, f"試したエンコーディング {encodings} すべてで失敗: {last_err}")

def make_town_base(t):
    """
    町名の「（…）」や「一丁目〜九丁目／1丁目〜9丁目」などを落としてベース名を作る
    例:
      上津台一丁目 → 上津台
      大通西（１〜１９丁目） → 大通西
    """
    t = str(t)
    t = re.sub(r"（.*?）", "", t)                     # （〜）を削除
    t = re.sub(r"([一二三四五六七八九十〇零0-9]+)丁目$", "", t)  # ○丁目を削除
    return t.strip()

def build_alpha_shape(coords, target_ratio=0.1):
    """
    点集合 coords ([(lon, lat), ...]) から concave hull を作る。
    - convex hull 面積に対する比率 target_ratio に近い α をスキャン
    - 線や点になった場合は薄く buffer して面に変換
    - 失敗した場合は凸包にフォールバック
    """
    if len(coords) < 3:
        return None

    mp_all = MultiPoint(coords)
    convex = mp_all.convex_hull
    convex_area = convex.area
    if convex_area == 0:
        poly = convex
    else:
        alpha_candidates = np.logspace(-2.3, 1, 24)  # ≒ 0.005〜10
        best_alpha = None
        best_shape = None
        best_diff = float("inf")

        for a in alpha_candidates:
            try:
                shp = alphashape.alphashape(coords, a)
            except Exception:
                continue
            if shp.is_empty or shp.area == 0:
                continue

            area = shp.area
            ratio = area / convex_area
            diff = abs(ratio - target_ratio)
            if diff < best_diff:
                best_diff = diff
                best_alpha = a
                best_shape = shp

        print("  chosen alpha:", best_alpha, " diff_from_target=", best_diff)
        poly = best_shape if best_shape is not None else convex

    # ポリゴン以外（線・点など）の場合は、薄く buffer して面にする
    if poly.geom_type in ("LineString", "MultiLineString", "Point", "MultiPoint"):
        poly = poly.buffer(1e-4)

    return poly

# ---------- 位置情報マスタの準備（utf_ken_all + ○○位置.csv） ----------

# utf_ken_all 読み込み
utf = read_csv_auto("utf_ken_all.csv", header_opt=None)

if utf.shape[1] < 9:
    raise ValueError("utf_ken_all.csv の列数が想定外です（KEN_ALL形式か確認してください）。")

utf.columns = [
    "jis", "old_zip", "zip7",
    "pref_kana", "city_kana", "town_kana",
    "pref", "city", "town"
] + [f"extra_{i}" for i in range(utf.shape[1] - 9)]

utf["zip7"] = utf["zip7"].astype(str).str.zfill(7)
utf_use = utf[["zip7", "pref", "city", "town"]].copy()
for c in ["pref", "city", "town"]:
    utf_use[c] = utf_use[c].astype(str).str.strip()
utf_use["town_base"] = utf_use["town"].map(make_town_base)

# ○○位置.csv を全部読み込んで全国マスタ作成
loc_list = []
for path in glob.glob("*位置*.csv"):
    print(f"位置参照ファイル候補: {path}")
    loc = read_csv_auto(path, header_opt=0)

    required = {"都道府県名", "市区町村名", "大字町丁目名", "緯度", "経度"}
    if not required.issubset(loc.columns):
        print(f"  {path} は必須列が足りないのでスキップ: {loc.columns}")
        continue

    df_pos = loc.copy()
    df_pos = df_pos.rename(columns={
        "都道府県名": "pref",
        "市区町村名": "city",
        "大字町丁目名": "town",
        "緯度": "lat",
        "経度": "lon",
    })

    for c in ["pref", "city", "town"]:
        df_pos[c] = df_pos[c].astype(str).str.strip()
    df_pos["town_base"] = df_pos["town"].map(make_town_base)

    loc_list.append(df_pos[["pref", "city", "town_base", "lat", "lon"]])

if not loc_list:
    raise ValueError("『○○位置.csv』が見つからないか、必須列（都道府県名/市区町村名/大字町丁目名/緯度/経度）がありません。")

loc_all = pd.concat(loc_list, ignore_index=True).dropna(subset=["lat", "lon"])

# 町名ベースごとに重心（平均）をとる
loc_base = (
    loc_all
    .groupby(["pref", "city", "town_base"], as_index=False)
    .agg(lat=("lat", "mean"), lon=("lon", "mean"))
)

# ---------- 会場の緯度経度を VENUE_ZIP から取得 ----------

venue_zip7 = re.sub(r"\D", "", VENUE_ZIP).zfill(7)

venue_addr = utf_use[utf_use["zip7"] == venue_zip7].copy()
if venue_addr.empty:
    raise ValueError(f"会場郵便番号 {venue_zip7} が utf_ken_all.csv に見つかりません。")

venue_addr["town_base"] = venue_addr["town"].map(make_town_base)
venue_addr = venue_addr.merge(loc_base, on=["pref", "city", "town_base"], how="left")
venue_addr = venue_addr.dropna(subset=["lat", "lon"])

if venue_addr.empty:
    raise ValueError(f"会場郵便番号 {venue_zip7} に対応する緯度経度が ○○位置.csv から取得できません。")

venue_lat = float(venue_addr.iloc[0]["lat"])
venue_lon = float(venue_addr.iloc[0]["lon"])
venue_point = Point(venue_lon, venue_lat)

print(f"会場座標: zip={venue_zip7}, lat={venue_lat}, lon={venue_lon}")

# ---------- 各回の90％商圏を計算 ----------

all_event_90_list = []   # 各回の90％点 DataFrame
event_polygons = []      # 各回の90％面ポリゴン

for idx, csv_path in enumerate(VISITOR_CSV_LIST, start=1):
    print(f"\n=== イベント{idx}: {csv_path} を処理中 ===")
    vis = read_csv_auto(csv_path, header_opt=None)
    vis.columns = [f"col_{i}" for i in range(vis.shape[1])]

    # A列 → 郵便番号
    vis["zip7"] = (
        vis["col_0"]
        .astype(str)
        .str.replace(r"\D", "", regex=True)
        .str.zfill(7)
    )
    vis = vis[vis["zip7"].str.match(r"^\d{7}$")].copy()
    if vis.empty:
        print("  有効な郵便番号が1件もないためスキップ")
        continue

    # 郵便番号 → pref/city/town
    vis_addr = vis.merge(utf_use, on="zip7", how="left")
    if vis_addr[["pref", "city", "town"]].isna().all(axis=None):
        print("  utf_ken_all にマッチする郵便番号がないためスキップ")
        continue

    # 緯度経度付与
    vis_addr["town_base"] = vis_addr["town"].map(make_town_base)
    vis_geo = vis_addr.merge(loc_base, on=["pref", "city", "town_base"], how="left")
    vis_geo = vis_geo.dropna(subset=["lat", "lon"]).copy()
    if vis_geo.empty:
        print("  緯度経度が1件も付かなかったためスキップ")
        continue

    vis_geo["lat"] = vis_geo["lat"].astype(float)
    vis_geo["lon"] = vis_geo["lon"].astype(float)

    # 会場からの距離（度単位・順位付け用）
    vis_geo["geom"] = [
        Point(lo, la) for lo, la in zip(vis_geo["lon"], vis_geo["lat"])
    ]
    vis_geo["dist"] = vis_geo["geom"].apply(lambda p: p.distance(venue_point))

    # 近い順に並べて上位90％を残す
    vis_geo_sorted = vis_geo.sort_values("dist").reset_index(drop=True)
    n = len(vis_geo_sorted)
    keep_n = math.ceil(n * 0.9)
    vis_90 = vis_geo_sorted.iloc[:keep_n].copy()
    vis_90["event_id"] = idx

    print(f"  総レコード数 {n} → 90％残し {keep_n} 件")

    # この回の90％点で concave hull を作る
    coords = list(zip(vis_90["lon"], vis_90["lat"]))
    print("  concave hull 計算中（イベントごと）...")
    hull = build_alpha_shape(coords, target_ratio=0.1)
    event_polygons.append((idx, hull))

    all_event_90_list.append(vis_90)

if not all_event_90_list:
    raise ValueError("有効なイベントデータが1件もありません。VISITOR_CSV_LIST とファイル内容を確認してください。")

# ---------- n回分90％データを統合 → 1人しかいない郵便番号を除外 ----------

all_90 = pd.concat(all_event_90_list, ignore_index=True)

# 郵便番号ごとの合計レコード数
zip_counts = all_90.groupby("zip7")["zip7"].transform("size")

# 合計1レコードしかない郵便番号を除外
mask_multi = zip_counts >= 2
removed_single = (~mask_multi).sum()
core_base = all_90[mask_multi].copy()

print(f"\n統合90％データ総数: {len(all_90)}")
print(f"『n回合計で1レコードしかない郵便番号』除外数: {removed_single}")
print(f"残りレコード数: {len(core_base)}")

if core_base.empty:
    raise ValueError("1回しか出てこない郵便番号を除いた結果、データが空になりました。")

# ---------- 統合データから遠い10％をカット ----------

core_base_sorted = core_base.sort_values("dist").reset_index(drop=True)
N_core = len(core_base_sorted)
keep_core_n = math.ceil(N_core * 0.9)  # 近い90％だけ残す
core_final = core_base_sorted.iloc[:keep_core_n].copy()

print(f"統合ベース {N_core} → 遠い10％カット後 {keep_core_n} 件")

# concave hull（統合コア商圏）
core_coords = list(zip(core_final["lon"], core_final["lat"]))
print("統合コア商圏 concave hull 計算中...")
core_hull = build_alpha_shape(core_coords, target_ratio=0.1)

# ---------- KML 出力（各回90％面＋統合コア面＋会場ピン＋点群） ----------

kml = simplekml.Kml()

# 会場マーカー
venue_pt = kml.newpoint(
    name=f"Venue_{venue_zip7}",
    coords=[(venue_lon, venue_lat)],
)
venue_pt.style.iconstyle.scale = 1.2
venue_pt.style.iconstyle.color = simplekml.Color.red

# 各回の90％面（薄めの色）
for idx, hull in event_polygons:
    if hull is None or hull.is_empty:
        continue

    # MultiPolygon の場合はいちばん大きい面
    if hull.geom_type == "MultiPolygon":
        hull = max(list(hull.geoms), key=lambda g: g.area)
    # 念のためポリゴン以外ならbufferで面化
    if hull.geom_type != "Polygon":
        hull = hull.buffer(1e-4)

    coords = list(hull.exterior.coords)

    poly = kml.newpolygon(
        name=f"event_{idx}_90pct_area",
        outerboundaryis=[(lon, lat) for lon, lat in coords],
    )
    poly.style.polystyle.color = simplekml.Color.changealpha('3f', simplekml.Color.blue)
    poly.style.polystyle.fill = 1
    poly.style.polystyle.outline = 1

# 各回90％点（フォルダごと）
for idx, vis_90 in enumerate(all_event_90_list, start=1):
    folder_pts = kml.newfolder(name=f"event_{idx}_90pct_points")
    for _, row in vis_90.iterrows():
        p = folder_pts.newpoint(
            coords=[(row["lon"], row["lat"])],
        )
        p.name = str(row.get("zip7", ""))
        p.style.iconstyle.scale = 0.5
        p.style.iconstyle.color = simplekml.Color.white

# 統合コア商圏（濃い紫）
if core_hull is not None and not core_hull.is_empty:
    if core_hull.geom_type == "MultiPolygon":
        core_hull = max(list(core_hull.geoms), key=lambda g: g.area)
    if core_hull.geom_type != "Polygon":
        core_hull = core_hull.buffer(1e-4)

    coords = list(core_hull.exterior.coords)
    poly_core = kml.newpolygon(
        name="core_merged_90pct_area",
        outerboundaryis=[(lon, lat) for lon, lat in coords],
    )
    poly_core.style.polystyle.color = simplekml.Color.changealpha('7f', simplekml.Color.purple)
    poly_core.style.polystyle.fill = 1
    poly_core.style.polystyle.outline = 1

# 統合コア点群
folder_core_pts = kml.newfolder(name="core_merged_90pct_points")
for _, row in core_final.iterrows():
    p = folder_core_pts.newpoint(
        coords=[(row["lon"], row["lat"])],
    )
    p.name = str(row.get("zip7", ""))
    p.style.iconstyle.scale = 0.7
    p.style.iconstyle.color = simplekml.Color.yellow

kml.save(OUTPUT_KML)
print(f"\n完了：{OUTPUT_KML} を出力しました。")
print("  ・各回の90％商圏（event_1_90pct_area, event_2_90pct_area, ...）")
print("  ・n回統合コア商圏（core_merged_90pct_area）")


utf_ken_all.csv を encoding='utf-8-sig' で読み込み成功
位置参照ファイル候補: 北海道位置.csv
北海道位置.csv を encoding='cp932' で読み込み成功
会場座標: zip=0600032, lat=43.0668631, lon=141.372593

=== イベント1: AJ-41-10 4w-サッポロファクトリーホール-新規対象来場393／1255.csv を処理中 ===
AJ-41-10 4w-サッポロファクトリーホール-新規対象来場393／1255.csv を encoding='cp932' で読み込み成功
  総レコード数 350 → 90％残し 315 件
  concave hull 計算中（イベントごと）...
  chosen alpha: 10.0  diff_from_target= 0.22794859933737352

=== イベント2: AJ-41-4-1w-サッポロファクトリー-新規対象来場413／1177.csv を処理中 ===
AJ-41-4-1w-サッポロファクトリー-新規対象来場413／1177.csv を encoding='cp932' で読み込み成功
  総レコード数 369 → 90％残し 333 件
  concave hull 計算中（イベントごと）...
  chosen alpha: 10.0  diff_from_target= 0.05607167716351036

=== イベント3: AJ-41-7-3w-サッポロファクトリーホール-新規対象来場480／1421.csv を処理中 ===
AJ-41-7-3w-サッポロファクトリーホール-新規対象来場480／1421.csv を encoding='cp932' で読み込み成功
  総レコード数 439 → 90％残し 396 件
  concave hull 計算中（イベントごと）...
  chosen alpha: 10.0  diff_from_target= 0.22592044357432942

=== イベント4: AJ-42-4 3w-サッポロファクトリーホール-新規対象来場286／985.csv を処理中 ===
AJ-42-4 3w-サッポロファクトリーホール-

In [ ]:
# ===============================================
# 郵便番号マッピング → 複数回開催 n 回分の統合コア商圏（○○位置.csv 版）
# 変更点：
#  - 90%カット → 2σカット（平均+2σ 以内を残す）
#  - 出力を「点KML」「面KML」に分割
# ===============================================

!pip -q install simplekml shapely alphashape

# ===== 設定（ここだけ自分の環境に合わせて書き換え） =====
VISITOR_CSV_LIST = [
    "AJ-41-10 4w-サッポロファクトリーホール-新規対象来場393／1255.csv",
    "AJ-42-4 3w-サッポロファクトリーホール-新規対象来場286／985.csv",
    "AJ-42-7 3w-サッポロファクトリーホール-神絵祭-華-新規対象来場575／2043.csv"
]
VENUE_ZIP = "060-0032"  # 会場郵便番号（ハイフンあり/なしOK）

# 出力（面KML / 点KML）
OUTPUT_KML_POLY   = "サッポロファクトリーホール_面_2sigma.kml"
OUTPUT_KML_POINTS = "サッポロファクトリーホール_点_2sigma.kml"

# ===== ここから下は触らなくてOK =====
import pandas as pd
import math, re, glob, os, zipfile
import numpy as np
import simplekml
import alphashape
from shapely.geometry import Point, MultiPoint

# ---------- 共通関数 ----------

def read_csv_auto(path, header_opt):
    encodings = ["cp932", "utf-8-sig", "utf-8"]
    last_err = None
    for enc in encodings:
        try:
            df = pd.read_csv(path, encoding=enc, header=header_opt)
            print(f"{path} を encoding='{enc}' で読み込み成功")
            return df
        except UnicodeDecodeError as e:
            last_err = e
            continue
        except FileNotFoundError as e:
            raise e
    raise UnicodeDecodeError(f"{path}", b"", 0, 1, f"試したエンコーディング {encodings} すべてで失敗: {last_err}")

def make_town_base(t):
    t = str(t)
    t = re.sub(r"（.*?）", "", t)
    t = re.sub(r"([一二三四五六七八九十〇零0-9]+)丁目$", "", t)
    return t.strip()

def haversine_m(lon1, lat1, lon2, lat2):
    # 地球半径（m）
    R = 6371000.0
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlmb = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlmb/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c

def sigma_cut(df, dist_col="dist_m", k=2.0):
    """
    平均+ k*標準偏差 以内だけ残す（遠い外れを落とす）
    """
    if df.empty:
        return df, None, None, None
    mu = float(df[dist_col].mean())
    sd = float(df[dist_col].std(ddof=0))  # 母標準偏差
    thr = mu + k * sd
    out = df[df[dist_col] <= thr].copy()
    return out, mu, sd, thr

def build_alpha_shape(coords, target_ratio=0.1):
    if len(coords) < 3:
        return None

    mp_all = MultiPoint(coords)
    convex = mp_all.convex_hull
    convex_area = convex.area

    if convex_area == 0:
        poly = convex
    else:
        alpha_candidates = np.logspace(-2.3, 1, 24)  # ≒ 0.005〜10
        best_alpha = None
        best_shape = None
        best_diff = float("inf")

        for a in alpha_candidates:
            try:
                shp = alphashape.alphashape(coords, a)
            except Exception:
                continue
            if shp.is_empty or getattr(shp, "area", 0) == 0:
                continue

            ratio = shp.area / convex_area
            diff = abs(ratio - target_ratio)
            if diff < best_diff:
                best_diff = diff
                best_alpha = a
                best_shape = shp

        print("  chosen alpha:", best_alpha, " diff_from_target=", best_diff)
        poly = best_shape if best_shape is not None else convex

    if poly.geom_type in ("LineString", "MultiLineString", "Point", "MultiPoint"):
        poly = poly.buffer(1e-4)

    return poly

def hull_to_polygon(hull):
    if hull is None or hull.is_empty:
        return None
    if hull.geom_type == "MultiPolygon":
        hull = max(list(hull.geoms), key=lambda g: g.area)
    if hull.geom_type != "Polygon":
        hull = hull.buffer(1e-4)
    if hull.geom_type != "Polygon" or hull.is_empty:
        return None
    return hull

def add_polygon(kml_obj, hull_poly, name, color_aabbggrr):
    coords = list(hull_poly.exterior.coords)
    poly = kml_obj.newpolygon(
        name=name,
        outerboundaryis=[(lon, lat) for lon, lat in coords],
    )
    poly.style.polystyle.color = color_aabbggrr
    poly.style.polystyle.fill = 1
    poly.style.polystyle.outline = 1
    return poly

# ---------- utf_ken_all の用意（zipがあれば展開） ----------
if (not os.path.exists("utf_ken_all.csv")) and os.path.exists("utf_ken_all.zip"):
    with zipfile.ZipFile("utf_ken_all.zip") as z:
        z.extractall(".")
    print("utf_ken_all.zip を展開しました")

if not os.path.exists("utf_ken_all.csv"):
    raise FileNotFoundError("utf_ken_all.csv（または utf_ken_all.zip）が見つかりません。")

# ---------- 位置情報マスタ（utf_ken_all + ○○位置.csv） ----------
utf = read_csv_auto("utf_ken_all.csv", header_opt=None)
if utf.shape[1] < 9:
    raise ValueError("utf_ken_all.csv の列数が想定外です（KEN_ALL形式か確認してください）。")

utf.columns = [
    "jis", "old_zip", "zip7",
    "pref_kana", "city_kana", "town_kana",
    "pref", "city", "town"
] + [f"extra_{i}" for i in range(utf.shape[1] - 9)]

utf["zip7"] = utf["zip7"].astype(str).str.zfill(7)
utf_use = utf[["zip7", "pref", "city", "town"]].copy()
for c in ["pref", "city", "town"]:
    utf_use[c] = utf_use[c].astype(str).str.strip()
utf_use["town_base"] = utf_use["town"].map(make_town_base)

loc_list = []
for path in glob.glob("*位置*.csv"):
    print(f"位置参照ファイル候補: {path}")
    loc = read_csv_auto(path, header_opt=0)

    required = {"都道府県名", "市区町村名", "大字町丁目名", "緯度", "経度"}
    if not required.issubset(loc.columns):
        print(f"  {path} は必須列が足りないのでスキップ: {loc.columns}")
        continue

    df_pos = loc.rename(columns={
        "都道府県名": "pref",
        "市区町村名": "city",
        "大字町丁目名": "town",
        "緯度": "lat",
        "経度": "lon",
    }).copy()

    for c in ["pref", "city", "town"]:
        df_pos[c] = df_pos[c].astype(str).str.strip()

    df_pos["town_base"] = df_pos["town"].map(make_town_base)
    loc_list.append(df_pos[["pref", "city", "town_base", "lat", "lon"]])

if not loc_list:
    raise ValueError("『○○位置.csv』が見つからないか、必須列（都道府県名/市区町村名/大字町丁目名/緯度/経度）がありません。")

loc_all = pd.concat(loc_list, ignore_index=True).dropna(subset=["lat", "lon"])
loc_base = (
    loc_all
    .groupby(["pref", "city", "town_base"], as_index=False)
    .agg(lat=("lat", "mean"), lon=("lon", "mean"))
)

# ---------- 会場座標 ----------
venue_zip7 = re.sub(r"\D", "", VENUE_ZIP).zfill(7)
venue_addr = utf_use[utf_use["zip7"] == venue_zip7].copy()
if venue_addr.empty:
    raise ValueError(f"会場郵便番号 {venue_zip7} が utf_ken_all.csv に見つかりません。")

venue_addr["town_base"] = venue_addr["town"].map(make_town_base)
venue_addr = venue_addr.merge(loc_base, on=["pref", "city", "town_base"], how="left").dropna(subset=["lat", "lon"])
if venue_addr.empty:
    raise ValueError(f"会場郵便番号 {venue_zip7} に対応する緯度経度が ○○位置.csv から取得できません。")

venue_lat = float(venue_addr.iloc[0]["lat"])
venue_lon = float(venue_addr.iloc[0]["lon"])
print(f"会場座標: zip={venue_zip7}, lat={venue_lat}, lon={venue_lon}")

# ---------- 各回：2σカット点集合 + 面 ----------
all_event_keep_list = []
event_polygons = []  # (event_id, polygon)

for idx, csv_path in enumerate(VISITOR_CSV_LIST, start=1):
    print(f"\n=== イベント{idx}: {csv_path} ===")
    vis = read_csv_auto(csv_path, header_opt=None)
    vis.columns = [f"col_{i}" for i in range(vis.shape[1])]

    vis["zip7"] = (
        vis["col_0"].astype(str)
        .str.replace(r"\D", "", regex=True)
        .str.zfill(7)
    )
    vis = vis[vis["zip7"].str.match(r"^\d{7}$")].copy()
    if vis.empty:
        print("  有効郵便番号なし → スキップ")
        continue

    vis_addr = vis.merge(utf_use, on="zip7", how="left")
    if vis_addr[["pref", "city", "town"]].isna().all(axis=None):
        print("  utf_ken_all にマッチなし → スキップ")
        continue

    vis_addr["town_base"] = vis_addr["town"].map(make_town_base)
    vis_geo = vis_addr.merge(loc_base, on=["pref", "city", "town_base"], how="left").dropna(subset=["lat", "lon"]).copy()
    if vis_geo.empty:
        print("  緯度経度が付かない → スキップ")
        continue

    vis_geo["lat"] = vis_geo["lat"].astype(float)
    vis_geo["lon"] = vis_geo["lon"].astype(float)

    # 距離（m）
    vis_geo["dist_m"] = [
        haversine_m(venue_lon, venue_lat, lo, la)
        for lo, la in zip(vis_geo["lon"], vis_geo["lat"])
    ]

    n0 = len(vis_geo)
    kept, mu, sd, thr = sigma_cut(vis_geo, dist_col="dist_m", k=2.0)
    n1 = len(kept)
    print(f"  レコード数 {n0} → 2σ(平均+2σ)以内 {n1}（しきい値={thr/1000:.2f}km, 平均={mu/1000:.2f}km, σ={sd/1000:.2f}km）")

    if kept.empty:
        print("  2σ後に0件 → スキップ")
        continue

    kept["event_id"] = idx

    # この回の面
    coords = list(zip(kept["lon"], kept["lat"]))
    print("  concave hull 計算中（イベントごと）...")
    hull = build_alpha_shape(coords, target_ratio=0.1)
    hull_poly = hull_to_polygon(hull)
    event_polygons.append((idx, hull_poly))

    all_event_keep_list.append(kept)

if not all_event_keep_list:
    raise ValueError("有効なイベントデータが1件もありません。VISITOR_CSV_LIST とファイル内容を確認してください。")

# ---------- n回分を統合 → 『合計1回しか出ない郵便番号』を除外 ----------
all_keep = pd.concat(all_event_keep_list, ignore_index=True)

zip_counts = all_keep.groupby("zip7")["zip7"].transform("size")
mask_multi = zip_counts >= 2
removed_single = int((~mask_multi).sum())
core_base = all_keep[mask_multi].copy()

print(f"\n統合（各回2σ後）レコード総数: {len(all_keep)}")
print(f"『n回合計で1レコードしかない郵便番号』除外数: {removed_single}")
print(f"残りレコード数: {len(core_base)}")

if core_base.empty:
    raise ValueError("1回しか出てこない郵便番号を除いた結果、データが空になりました。")

# ---------- 統合データを 2σカット（遠い外れを落とす） ----------
core_final, mu2, sd2, thr2 = sigma_cut(core_base, dist_col="dist_m", k=2.0)
print(f"統合ベース {len(core_base)} → 統合2σ以内 {len(core_final)}（しきい値={thr2/1000:.2f}km, 平均={mu2/1000:.2f}km, σ={sd2/1000:.2f}km）")

if core_final.empty:
    raise ValueError("統合2σカット後にデータが0件になりました。")

# 統合コア商圏（面）
core_coords = list(zip(core_final["lon"], core_final["lat"]))
print("統合コア商圏 concave hull 計算中...")
core_hull = build_alpha_shape(core_coords, target_ratio=0.1)
core_hull_poly = hull_to_polygon(core_hull)

# ---------- KML：面（ポリゴン） ----------
kml_poly = simplekml.Kml()

# 会場ピン（面KMLにも入れる）
venue_pt_poly = kml_poly.newpoint(name=f"Venue_{venue_zip7}", coords=[(venue_lon, venue_lat)])
venue_pt_poly.style.iconstyle.scale = 1.2
venue_pt_poly.style.iconstyle.color = simplekml.Color.red

# 各回の面（薄青）
for idx, hull_poly in event_polygons:
    if hull_poly is None:
        continue
    add_polygon(
        kml_poly,
        hull_poly,
        name=f"event_{idx}_2sigma_area",
        color_aabbggrr=simplekml.Color.changealpha('3f', simplekml.Color.blue)
    )

# 統合コア面（濃紫）
if core_hull_poly is not None:
    add_polygon(
        kml_poly,
        core_hull_poly,
        name="core_merged_2sigma_area",
        color_aabbggrr=simplekml.Color.changealpha('7f', simplekml.Color.purple)
    )

kml_poly.save(OUTPUT_KML_POLY)
print(f"\n面KML 出力完了：{OUTPUT_KML_POLY}")

# ---------- KML：点（ポイント） ----------
kml_pts = simplekml.Kml()

# 会場ピン（点KMLにも入れる）
venue_pt_pts = kml_pts.newpoint(name=f"Venue_{venue_zip7}", coords=[(venue_lon, venue_lat)])
venue_pt_pts.style.iconstyle.scale = 1.2
venue_pt_pts.style.iconstyle.color = simplekml.Color.red

# 各回の点（フォルダ）
for i, kept in enumerate(all_event_keep_list, start=1):
    folder = kml_pts.newfolder(name=f"event_{i}_2sigma_points")
    for _, row in kept.iterrows():
        p = folder.newpoint(coords=[(float(row["lon"]), float(row["lat"]))])
        p.name = str(row.get("zip7", ""))
        p.style.iconstyle.scale = 0.5
        p.style.iconstyle.color = simplekml.Color.white

# 統合コア点（黄）
folder_core = kml_pts.newfolder(name="core_merged_2sigma_points")
for _, row in core_final.iterrows():
    p = folder_core.newpoint(coords=[(float(row["lon"]), float(row["lat"]))])
    p.name = str(row.get("zip7", ""))
    p.style.iconstyle.scale = 0.7
    p.style.iconstyle.color = simplekml.Color.yellow

kml_pts.save(OUTPUT_KML_POINTS)
print(f"点KML 出力完了：{OUTPUT_KML_POINTS}")

print("\n完了：")
print("  ・各回（2σ）面 + 統合コア面 →", OUTPUT_KML_POLY)
print("  ・各回（2σ）点 + 統合コア点 →", OUTPUT_KML_POINTS)


utf_ken_all.csv を encoding='utf-8-sig' で読み込み成功
位置参照ファイル候補: 北海道位置.csv
北海道位置.csv を encoding='cp932' で読み込み成功
会場座標: zip=0600032, lat=43.0668631, lon=141.372593

=== イベント1: AJ-41-10 4w-サッポロファクトリーホール-新規対象来場393／1255.csv ===
AJ-41-10 4w-サッポロファクトリーホール-新規対象来場393／1255.csv を encoding='cp932' で読み込み成功
  レコード数 350 → 2σ(平均+2σ)以内 335（しきい値=87.43km, 平均=18.55km, σ=34.44km）
  concave hull 計算中（イベントごと）...
  chosen alpha: 7.186571307235264  diff_from_target= 0.020908701405034605

=== イベント2: AJ-42-4 3w-サッポロファクトリーホール-新規対象来場286／985.csv ===
AJ-42-4 3w-サッポロファクトリーホール-新規対象来場286／985.csv を encoding='cp932' で読み込み成功
  レコード数 254 → 2σ(平均+2σ)以内 240（しきい値=66.11km, 平均=15.47km, σ=25.32km）
  concave hull 計算中（イベントごと）...
  chosen alpha: 10.0  diff_from_target= 0.12679455133777326

=== イベント3: AJ-42-7 3w-サッポロファクトリーホール-神絵祭-華-新規対象来場575／2043.csv ===
AJ-42-7 3w-サッポロファクトリーホール-神絵祭-華-新規対象来場575／2043.csv を encoding='cp932' で読み込み成功
  レコード数 531 → 2σ(平均+2σ)以内 508（しきい値=80.72km, 平均=15.89km, σ=32.41km）
  concave hull 計算中（イベントごと）...
  chosen alpha